In [1]:
import numpy as np
import random

from rnn import RNN
from data import train_data, test_data

In [2]:
# Create the vocabulary.
vocab = list(set([w for text in train_data.keys() for w in text.split(' ')]))
vocab_size = len(vocab)
print('%d unique words found' % vocab_size)

18 unique words found


In [3]:
# Assign indices to each word.
word_to_idx = { w: i for i, w in enumerate(vocab) }
idx_to_word = { i: w for i, w in enumerate(vocab) }

In [4]:
# Initialize our RNN!
rnn = RNN(vocab_size, 2)

In [5]:
def createInputs(text):
  '''
  Returns an array of one-hot vectors representing the words in the input text string.
  - text is a string
  - Each one-hot vector has shape (vocab_size, 1)
  '''
  inputs = []
  for w in text.split(' '):
    v = np.zeros((vocab_size, 1))
    v[word_to_idx[w]] = 1
    inputs.append(v)
  return inputs

In [6]:
def softmax(xs):
  # Applies the Softmax Function to the input array.
  return np.exp(xs) / sum(np.exp(xs))

In [7]:
def processData(data, backprop=True):
  '''
  Returns the RNN's loss and accuracy for the given data.
  - data is a dictionary mapping text to True or False.
  - backprop determines if the backward phase should be run.
  '''
  items = list(data.items())
  random.shuffle(items)

  loss = 0
  num_correct = 0

  for x, y in items:
    inputs = createInputs(x)
    target = int(y)

    # Forward
    out, _ = rnn.forward(inputs)
    probs = softmax(out)

    # Calculate loss / accuracy
    loss -= np.log(probs[target])
    num_correct += int(np.argmax(probs) == target)

    if backprop:
      # Build dL/dy
      d_L_d_y = probs
      d_L_d_y[target] -= 1

      # Backward
      rnn.backprop(d_L_d_y)

  return loss / len(data), num_correct / len(data)

In [8]:
# Training loop
for epoch in range(1000):
  train_loss, train_acc = processData(train_data)

  if epoch % 100 == 99:
    print('--- Epoch %d' % (epoch + 1))
    print('Train:\tLoss %.3f | Accuracy: %.3f' % (train_loss, train_acc))

    test_loss, test_acc = processData(test_data, backprop=False)
    print('Test:\tLoss %.3f | Accuracy: %.3f' % (test_loss, test_acc))

--- Epoch 100
Train:	Loss 0.688 | Accuracy: 0.569
Test:	Loss 0.698 | Accuracy: 0.500
--- Epoch 200
Train:	Loss 0.672 | Accuracy: 0.655
Test:	Loss 0.718 | Accuracy: 0.550
--- Epoch 300
Train:	Loss 0.216 | Accuracy: 0.931
Test:	Loss 0.118 | Accuracy: 1.000
--- Epoch 400
Train:	Loss 0.012 | Accuracy: 1.000
Test:	Loss 0.027 | Accuracy: 1.000
--- Epoch 500
Train:	Loss 0.685 | Accuracy: 0.690
Test:	Loss 0.697 | Accuracy: 0.500
--- Epoch 600
Train:	Loss 0.332 | Accuracy: 0.828
Test:	Loss 0.352 | Accuracy: 0.900
--- Epoch 700
Train:	Loss 0.178 | Accuracy: 0.914
Test:	Loss 0.327 | Accuracy: 0.900
--- Epoch 800
Train:	Loss 0.007 | Accuracy: 1.000
Test:	Loss 0.021 | Accuracy: 1.000
--- Epoch 900
Train:	Loss 0.004 | Accuracy: 1.000
Test:	Loss 0.015 | Accuracy: 1.000
--- Epoch 1000
Train:	Loss 0.002 | Accuracy: 1.000
Test:	Loss 0.012 | Accuracy: 1.000
